In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tqdm import tqdm
import os
from scipy.misc import imread,imresize
import inception_resnet_v2

In [9]:
checkpoint = 'inception_resnet_v2_2016_08_30.ckpt'

In [4]:
img_size = inception_resnet_v2.inception_resnet_v2.default_image_size
img_size

299

In [11]:
batch_size = 8
learning_rate = 1e-3
classes = 196

In [16]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
X = tf.placeholder(tf.float32,[None,img_size, img_size, 3])
Y = tf.placeholder(tf.int32, [None])

images = tf.map_fn(lambda image: tf.image.per_image_standardization(image), X)
with slim.arg_scope(inception_resnet_v2.inception_resnet_v2_arg_scope()):
    logits, endpoints = inception_resnet_v2.inception_resnet_v2(images)
logits = tf.layers.dense(logits, classes)

cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits=logits)
cost = tf.reduce_mean(cross_entropy)
accuracy = tf.reduce_mean(tf.cast(tf.nn.in_top_k(logits, Y, 1), tf.float32))

global_step = tf.Variable(0, name="global_step", trainable=False)

tf.summary.scalar("total_loss", cost)
tf.summary.scalar("accuracy", accuracy)

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost,global_step=global_step)

sess.run(tf.global_variables_initializer())
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'InceptionResnetV2')
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, checkpoint)

INFO:tensorflow:Restoring parameters from inception_resnet_v2_2016_08_30.ckpt
